# Recognition - Classification

### Imports et installations

In [ ]:
import numpy as np
#!pip install -q keras
import keras

### Mount your drive

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!ls
print()
!ls drive
print()
!ls drive/My\ Drive

drive  sample_data

MyDrive

 1A
 20240427_50148593_1708442.pdf
 20240503_50320795_1708442.pdf
 20240609_51150808_1708442.pdf
 20240802_52601703_1708442.pdf
 20240830_53319313_1708442.pdf
 20241115_55314637_1708442.pdf
 2A
 397335-1293766.pdf
 Admin
 AI4MS_Summary_v2.pdf
 AI-seminar-gdPublic-ECM.gslides
 AI-seminar-gdPublic.gslides
'anglais charlie.gdoc'
 ANR-AAPG-2025-Trame_pre-proposal.docx
'AutoVP Sicre.docx'
 boarding-pass.pdf
 candidates_PHD_UnLIR.gsheet
 carte_feliciation_LucG_.pdf
'Centrale_Conducteur Intervenants Final.xlsx'
'Chiens des enfers.gdoc'
 Choix_projets_3A_23-24.gsheet
 choix_projets_3A_24-25.gsheet
 Chore_laure
 Classroom
'Colab Notebooks'
'Contact Information.gform'
'Copie AAE PROMESSE DE VENTE JULIEN _ SICRE.pdf'
'Copy of ANR-AAPG-2025-Trame_pre-proposal_v2.docx'
 correc_CV21-21
'CSC_23_Explainability for visual recognition deep learning models..gdoc'
 csi.docx
 csi_jenkins.docx
 CV_intro.gslides
'CVPR 2020_SIC.gslides'
'CVPR21 interpretability papers.gdoc'
 CV_sl

### Get the dataset

Copy the dataset on your drive. Specify the path to your dataset in the following codes ! You could use !wget as an alternative.


### List images from folders

In [ ]:
import os
#import glob

dir_path  = 'drive/MyDrive/PATH_TO_miniMIT'#####################################
listDir = sorted(os.listdir(dir_path))#glob.glob(dir_path)
print(listDir)

for d in listDir:
  #read subfolder
  listFiles = sorted(os.listdir(dir_path+'/'+d))
  print(listFiles)
  print(len(listFiles))


### Available networks

We are going to use available pre-trained networks, see  https://keras.io/applications/

We will first focus on VGG19.


### Create one batch including all training images

Note that images are preprocessed



In [ ]:

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, ActivityRegularization, Lambda
from keras.layers import Convolution2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.layers import AveragePooling2D, Input
######from keras.utils import np_utils
from keras.utils import normalize
from keras import backend as K

import os
import glob
import time

#import sklearn
#from sklearn.preprocessing import normalize

from tensorflow.keras.applications.vgg19 import VGG19
from keras.utils import load_img, img_to_array
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model


dir_path  = 'drive/MyDrive/PATH_TO_miniMIT/train'##############################""
listDir = sorted(os.listdir(dir_path))
print(len(listDir))



X_train = np.zeros(shape=(40*3,224,224,3), dtype=np.float32)# train batch

Y_train = np.zeros(shape=(40*3,3))
Y_train[0:40,0]=1
Y_train[40:80,1]=1
Y_train[80:120,2]=1


cpt = 0
for d in listDir:#[listDir[1],  listDir[2] ,listDir[4]]:
  #read subfolder
  listFiles = sorted(os.listdir(dir_path+'/'+d))
  print(d)
  print(len(listFiles))

  for f in listFiles:
    img = load_img(dir_path+'/'+d+'/'+f, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    #print(x.shape)
    X_train[cpt,:,:,:] = x[0,:,:,:]
    cpt+=1
    print(cpt)

 #Use np.save(...) function if you want to save your train batch

FileNotFoundError: [Errno 2] No such file or directory: 'drive/MyDrive/PATH_TO_miniMIT/train'

### Build Similarly one batch for the test data X_test and its labels Y_test

In [ ]:
#TODO

### Build a new network based on a pre-trained network

In [ ]:
network = VGG19(weights='imagenet')
print(network.summary())

network2 = Model(inputs=network.input, outputs=network.get_layer('block4_pool').output)#Use only the pre-trained convolutional layers
#######You may also use network with the option include_top = False
#print(network2.summary())
network2.trainable = False# this freezes the layers of this network, i.e. the layers will not be updated
print(network2.output_shape)


#Building a new network to plug after the first one.
input_shape = (14,14,512)

x = Input(shape=input_shape, name='input')
y0 = AveragePooling2D((14,14), padding='same')(x)# MaxPooling2D   AveragePooling2D ##### You may also use GlobalAveragePooling layers

y0 = keras.layers.Normalization(axis=-1)(y0)
y0 = Flatten()(y0)

y0 = Dense(3)(y0)##
y1 = Activation('softmax')(y0)

model1 = Model(inputs=x,outputs=y1)

#Combine the networks
modelf = Model(inputs=network2.input, outputs=model1(network2.output))
modelf.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
#print(modelf.summary())


574710816/574710816 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "vgg19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv4 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv4 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv4 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 143,667,240 (548.05 MB)

 Trainable params: 143,667,240 (548.05 MB)

 Non-trainable params: 0 (0.00 B)

None
(None, 14, 14, 512)


### Train the network

In [ ]:
print(X_train.shape)
print(Y_train.shape)

modelf.fit(X_train, Y_train, batch_size=40, epochs=50, verbose=1)
#modelf.fit(X_train, Y_train, validation_data= (X_test,Y_test), batch_size=40, epochs=50, verbose=1)


### Test the network on the test set

In [ ]:
#TODO

### Play with the architecture

> - Add 1 or 2 more fully connected layers (512 and 20 for exemple) with Relu and dropout ( https://keras.io/activations/ https://keras.io/layers/core/#dropout )

> - Switch the optimizer to adam ( https://keras.io/optimizers/ ). What do you observe ? why ?

> - Switch AveragePooling to MaxPooling

> - Replace l2 normalization with Bath norm.

> - Play with epochs and batch size

In [ ]:
#TODO

### Report the architectures tested and the performance obtained.

In [ ]:
#TODO

### Modify the network

> - Extract the first (then second) fully connected layer from the pre-trained vgg19 (There is no need for pooling and flatenning)

> - Compare your results to the convolutional layer


In [ ]:
#TODO

### Use another network

> - Choose another network, namely resnet, densenet, mobileNet, efficientNet...

> - Study its architecture

> - Similarly extract the final or penultimate convolutional layer, then add a couple of dense layers to perform classification. Compare performances with earlier networks.



In [ ]:
#TODO

### Play some more

> - re-compute your batch with larger images as input (twice as large for instance). Then use a fully convolutional architecture such as ResNet or VGG with the option include_top= False. Add a global average (or max) pooling layer (if necessary), followed by a dense classification layer.

In [ ]:
#TODO

##Optional

You may build or get your own dataset and perform classification on it.